In [1]:
import cv2
import numpy as np

# 读入图像
img = cv2.imread("D:/Game/upian/uling.png", cv2.IMREAD_GRAYSCALE)
cv2.imshow("Original Image", img)
# 生成高斯噪声
def generate_noise(img_size, sigma):
    noise = np.zeros(img_size, np.float32)
    for i in range(img_size[0]):
        for j in range(img_size[1]):
            u1 = np.random.rand()
            u2 = np.random.rand()
            z1 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
            noise[i, j] = sigma * z1
    return noise

# 加入高斯噪声
img_size = img.shape
noise_sigma_20 = generate_noise(img_size, 20)
noise_sigma_40 = generate_noise(img_size, 40)
img_noise_sigma_20 = img.astype(np.float32) + noise_sigma_20
img_noise_sigma_40 = img.astype(np.float32) + noise_sigma_40
cv2.imshow("Noisy Image (20dB)", np.uint8(img_noise_sigma_20))
cv2.imshow("Noisy Image (10dB)", np.uint8(img_noise_sigma_40))
# 维纳滤波
def wiener_filter(img_noise, psf, K):
    H = np.fft.fft2(psf, img_noise.shape)
    H_conj = np.conj(H)
    SNR = K
    if K == None:
        SNR = 0
        v_noise = np.var(img_noise)
        v_signal = np.var(img)
        SNR = v_signal / v_noise
    F = np.fft.fft2(img_noise)
    H_abs_squared = np.square(np.abs(H))
    G_hat = (H_conj / (H_abs_squared + SNR)) * F
    g_hat = np.real(np.fft.ifft2(G_hat))
    return g_hat
    # 评价指标：PSNR
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    pixel_max = 255.0
    return 20 * np.log10(pixel_max / np.sqrt(mse))

# 维纳滤波，信噪比未知
psf = np.ones((7, 7)) / 49
img_restore_sigma_20 = wiener_filter(img_noise_sigma_20, psf, None)
img_restore_sigma_40 = wiener_filter(img_noise_sigma_40, psf, None)
cv2.imshow("Restored Image (Wiener, unknown SNR, 20dB)", np.uint8(img_restore_sigma_20))
cv2.imshow("Restored Image (Wiener, unknown SNR, 10dB)", np.uint8(img_restore_sigma_40))
print("PSNR (Wiener, unknown SNR, 20dB): ", psnr(img, img_restore_sigma_20))
print("PSNR (Wiener, unknown SNR, 10dB): ", psnr(img, img_restore_sigma_40))

# 维纳滤波，信噪比已知
psf = np.ones((5, 5)) / 25
img_restore_sigma_20_known = wiener_filter(img_noise_sigma_20, psf, 100)
img_restore_sigma_40_known = wiener_filter(img_noise_sigma_40, psf, 10)
cv2.imshow("Restored Image (Wiener, known SNR, 20dB)", np.uint8(img_restore_sigma_20_known))
cv2.imshow("Restored Image (Wiener, known SNR, 10dB)", np.uint8(img_restore_sigma_40_known))
print("PSNR (Wiener, known SNR, 20dB): ", psnr(img, img_restore_sigma_20_known))
print("PSNR (Wiener, known SNR, 10dB): ", psnr(img, img_restore_sigma_40_known))

cv2.waitKey(0)
cv2.destroyAllWindows()

PSNR (Wiener, unknown SNR, 20dB):  11.58580947530229
PSNR (Wiener, unknown SNR, 10dB):  12.916937185398806
PSNR (Wiener, known SNR, 20dB):  5.334898310616668
PSNR (Wiener, known SNR, 10dB):  6.069978118372915
